In [1]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit

from parallel_two_qubit_gate_decomposition import *
from gates_numpy import cnot_gate, fsim_gate, cphase_gate, xy_gate, get_gate_unitary_qiskit



## Decomposition example with a single two-qubit gate 

In [2]:
# CNOT gate in the circuit
my_op = np.matrix([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,0,1],
                   [0,0,1,0]])

In [3]:
test_circuit = QuantumCircuit(2)
test_circuit.unitary(my_op, [0,1])

In [4]:
test_circuit.draw()

┌──────────┐
q_0: ┤0         ├
     │  Unitary │
q_1: ┤1         ├
     └──────────┘

In [5]:
# Assume the HW supports the Google SYC gate. This is an underlying fsim_gate. 
# So we pass the fsim_gate function in the gate_defs,
# we pass the desired SYC parameters (theta=pi/2, phi=pi/6) in the params, we pass a label for circuit drawing, 
# and we pass the calibration fidelity of this gate on qubits 0 and 1

fid_2q = {(0,1): [0.995]} 
params = [[np.pi/2, np.pi/6]]
gate_labels = ['SYC']
gate_defs = [fsim_gate]

# All these inputs get passed to the gate decomposition pass. 
# Ignore the [1 to 54] loop (that was from our assumption of Sycamore hardware, but I believe it no longer plays a role)
# tolerance defines the fidelity accuracy we want from the decomposition. If you set the tol=0.1 or so, you can see that 
# lesser two-qubit gates are used.

pgrp = ParallelGateReplacementPass(gate_defs, params, gate_labels,fid_2q, [1 for _ in range(54)], tol=0.01)
approx_cz = pgrp.run(test_circuit)

In [6]:
approx_cz.draw()

┌─────────────────────────┐ ┌──────┐┌─────────────────────────┐┌──────┐»
q_0: ┤ U3(4.0204,5.2871,4.357) ├─┤1     ├┤ U3(3.1416,1.4988,6.212) ├┤1     ├»
     ├─────────────────────────┴┐│  SYC │├─────────────────────────┤│  SYC │»
q_1: ┤ U3(6.2832,7.0451,4.8067) ├┤0     ├┤ U3(1.6427,5.048,5.7377) ├┤0     ├»
     └──────────────────────────┘└──────┘└─────────────────────────┘└──────┘»
«     ┌──────────────────────────┐
«q_0: ┤ U3(3.9059,1.9684,3.3621) ├
«     ├──────────────────────────┤
«q_1: ┤ U3(3.1416,4.2677,3.5538) ├
«     └──────────────────────────┘

## Decomposition example with two circuit gates with two hardware gates

In [7]:
test_circuit = QuantumCircuit(5)
test_circuit.unitary(my_op, [2,3])
test_circuit.unitary(my_op, [3,4])

In [8]:
fid_2q = {(2,3): [0.94, #CZ fidelity
                  0.99],#XY fidelity
          (3,4): [0.93, #CZ
                  0.96]}#XY
params = [[np.pi], 
          [np.pi]]
gate_labels = ['CZ', 
               'XY']
gate_defs = [cphase_gate, 
             xy_gate]
pgrp = ParallelGateReplacementPass(gate_defs, params, gate_labels,fid_2q, [1 for _ in range(54)], tol=0.00000001)
approx = pgrp.run(test_circuit)

In [9]:
approx.draw()

»
q_0: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
q_1: ─────────────────────────────────────────────────────────────────────────»
          ┌─────────────────────┐     ┌─────┐   ┌──────────────────────────┐  »
q_2: ─────┤ U3(4.7124,4.7913,π) ├─────┤1    ├───┤ U3(4.7124,4.2624,4.5596) ├──»
         ┌┴─────────────────────┴┐    │  XY │┌──┴──────────────────────────┴─┐»
q_3: ────┤ U3(π/2,1.7236,2.0344) ├────┤0    ├┤ U3(-5.8525e-07,6.1985,5.5563) ├»
     ┌───┴───────────────────────┴───┐└─────┘└───────────────────────────────┘»
q_4: ┤ U3(-2.3395e-06,1.6161,3.7951) ├────────────────────────────────────────»
     └───────────────────────────────┘                                        »
«                                                                           
«q_0: ──────────────────────────────────────────────────────────────────────
«                                                                           
«q_1: ──────────────────────────────────────────────────────────────────────
«     ┌─────┐   ┌──────────────────────┐                                    
«q_2: ┤1    ├───┤ U3(1.5708,π,0.73256) ├────────────────────────────────────
«     │  XY │┌──┴──────────────────────┴─┐┌─────┐ ┌───────────────────────┐ 
«q_3: ┤0    ├┤ U3(0.46364,3.2709,3.5916) ├┤1    ├─┤ U3(π/2,6.2832,1.4415) ├─
«     └─────┘└───────────────────────────┘│  CZ │┌┴───────────────────────┴┐
«q_4: ────────────────────────────────────┤0    ├┤ U3(6.2832,4.3232,2.832) ├
«                                         └─────┘└─────────────────────────┘

## Multi-qubit unitary example

In [2]:
from parallel_multi_qubit_gate_decomposition import *

In [3]:
test_circuit = QuantumCircuit(4)
my_op = np.kron(fsim_gate(np.pi/2, np.pi/6), fsim_gate(np.pi/3, np.pi/6)) #creating a test unitary by multiplying two fsim gates
test_circuit.unitary(my_op, [0,1,2,3])

In [12]:
test_circuit.draw()

┌──────────┐
q_0: ┤0         ├
     │          │
q_1: ┤1         ├
     │  Unitary │
q_2: ┤2         ├
     │          │
q_3: ┤3         ├
     └──────────┘

In [13]:
fid_2q = {(0,1): [0.99, 0.99],
          (1,2): [0.99, 0.99],
          (2,3): [0.94, #CZ fidelity
                  0.99],#XY fidelity
          (3,4): [0.93, #CZ
                  0.96]}#XY
params = [[np.pi], 
          [np.pi]]
gate_labels = ['CZ', 
               'XY']
gate_defs = [cphase_gate, 
             xy_gate]
pgrp = MultiQubitGateReplacementPass(gate_defs, params, gate_labels,fid_2q, [1 for _ in range(54)], tol=0.0001)
approx = pgrp.run(test_circuit)

0.9914448610463646
0.008555138953635355


In [14]:
approx.draw()

┌──────────────────────────┐                                           »
q_0: ┤ U3(1.5708,3.2587,3.7362) ├───────────────────────────────────────────»
     ├──────────────────────────┤                                           »
q_1: ┤ U3(3.4275,4.6731,1.3419) ├───────────────────────────────────────────»
     ├─────────────────────────┬┘┌─────┐┌───────────────────────────┐┌─────┐»
q_2: ┤ U3(1.8003,1.065,5.2354) ├─┤1    ├┤ U3(3.1416,1.6997,0.14697) ├┤1    ├»
     ├─────────────────────────┴┐│  XY │├───────────────────────────┤│  XY │»
q_3: ┤ U3(6.2832,4.2057,2.7951) ├┤0    ├┤ U3(6.0066,0.72829,2.3247) ├┤0    ├»
     └──────────────────────────┘└─────┘└───────────────────────────┘└─────┘»
«                                                                        »
«q_0: ───────────────────────────────────────────────────────────────────»
«                                 ┌─────┐    ┌─────────────────────┐     »
«q_1: ────────────────────────────┤1    ├────┤ U3(π,3.2916,5.8205) ├─────»
«     ┌──────────────────────────┐│  XY │┌───┴─────────────────────┴────┐»
«q_2: ┤ U3(3.5655,4.3968,5.9023) ├┤0    ├┤ U3(4.0879e-06,3.6402,2.9126) ├»
«     ├─────────────────────────┬┘└─────┘└──────────────────────────────┘»
«q_3: ┤ U3(6.2832,4.3756,1.458) ├────────────────────────────────────────»
«     └─────────────────────────┘                                        »
«                                       ┌─────┐┌─────────────────────────┐ »
«q_0: ──────────────────────────────────┤1    ├┤ U3(5.236,3.7791,3.7149) ├─»
«     ┌─────┐┌─────────────────────────┐│  XY │├─────────────────────────┴┐»
«q_1: ┤1    ├┤ U3(1.378,4.3514,0.4963) ├┤0    ├┤ U3(7.3304,2.0089,1.4537) ├»
«     │  XY │├─────────────────────────┤├─────┤└─┬─────────────────────┬──┘»
«q_2: ┤0    ├┤ U3(1.7051,2.4069,3.208) ├┤1    ├──┤ U3(π,1.6828,4.7007) ├───»
«     └─────┘└─────────────────────────┘│  XY │┌─┴─────────────────────┴──┐»
«q_3: ──────────────────────────────────┤0    ├┤ U3(3.2319,4.7668,4.4817) ├»
«                                       └─────┘└──────────────────────────┘»
«     ┌─────┐ ┌──────────────────────────┐ 
«q_0: ┤1    ├─┤ U3(4.7124,2.2852,2.7035) ├─
«     │  XY │┌┴──────────────────────────┴┐
«q_1: ┤0    ├┤ U3(4.7124,0.71435,0.93327) ├
«     └─────┘└────────────────────────────┘
«q_2: ─────────────────────────────────────
«                                          
«q_3: ─────────────────────────────────────
«

In [ ]:
fid_2q = {(0,1): [0.999, 0.999],
          (1,2): [0.999, 0.999],
          (2,3): [0.999, #CZ fidelity
                  0.999],#XY fidelity
          (3,4): [0.999, #CZ
                  0.999]}#XY
params = [[np.pi/2, np.pi/6], 
          [np.pi/3, np.pi/6]]
gate_labels = ['fsim', 
               'fsim']
gate_defs = [fsim_gate, 
             fsim_gate]
pgrp = MultiQubitGateReplacementPass(gate_defs, params, gate_labels,fid_2q, [1 for _ in range(54)], tol=0.0001)
approx = pgrp.run(test_circuit)